<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/main/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [ ]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [138]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn
import time

In [206]:
California = ee.Geometry.Rectangle(-124.18, 41.39, -114.35, 32.75)
earliestDataStart = '2014-09-01T00:00:00'
# Import our fire events as features
modis_fires = ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters').filterBounds(California).filter(ee.Filter.gte('InitialDate', ee.Date.millis(earliestDataStart)))
# filter for only fires with greater than 10,000,000 square meters
modis_fires = modis_fires.filter(ee.Filter.gt('area', 10000000)).randomColumn(seed=0)

# Add a band for if this is a really big fire or not. 
# NOTE: We are using fire size as a proxy for severity
massive_fire_threshold = 100000000
massive_fires = modis_fires.filter(ee.Filter.gt('area', massive_fire_threshold)).limit(10, 'random').map(lambda f: f.set({'massive': 1}))  # 158 fires
other_fires = modis_fires.filter(ee.Filter.lte('area', massive_fire_threshold)).limit(10, 'random').map(lambda f: f.set({'massive': 0}))   # 609 fires

# split the fires into a test and training set
massive_fires_test = massive_fires.limit(3, 'random', False)
massive_fires_train = massive_fires.limit(7, 'random', True)
other_fires_test = other_fires.limit(3, 'random', False)
other_fires_train = other_fires.limit(7, 'random', True)

modis_fires_train = massive_fires_train.merge(other_fires_train)
modis_fires_test = massive_fires_test.merge(other_fires_test)
modis_fires = modis_fires_train.merge(modis_fires_test)

In [207]:
print("massive", modis_fires.filter(ee.Filter.eq('massive', 1)).size().getInfo())
print("small", modis_fires.filter(ee.Filter.eq('massive', 0)).size().getInfo())

massive 10
small 10


In [208]:
slo = ee.Geometry.Point([-120.4358, 35.3102])

# Import ImageCollections
landfire_prs = ee.Image("LANDFIRE/Fire/PRS/v1_2_0/CONUS")
landfire_evc = ee.Image("LANDFIRE/Vegetation/EVC/v1_4_0/CONUS")
landfire_evh = ee.Image("LANDFIRE/Vegetation/EVH/v1_4_0/CONUS")
landfire_evt = ee.Image("LANDFIRE/Vegetation/EVT/v1_4_0/CONUS")
daily_temp = ee.ImageCollection("ECMWF/ERA5/DAILY").filterDate(earliestDataStart, '2021-01-01')

fire_image2 = modis_fires.reduceToImage(
    properties = ['InitialDate'], 
    reducer = ee.Reducer.first()).rename('InitialDate')

composite = landfire_evc.addBands(landfire_prs).addBands(landfire_evh).addBands(landfire_evt).addBands(fire_image2)

In [217]:
bands = ['PRS', 'EVC','EVH','EVT','mean_temp', 'wind_speed']
# split the data into training and test sets
fire_image = ee.Image().byte().paint(modis_fires, 'massive').rename('massive')
# fire_image2 = ee.Image().paint(modis_fires,'IntitalDate').rename('InitialDate')





#fire_image = ee.Image().constant(modis_fires.first().toDictionary(props).values()).rename(props)

#fire_image = ee.ImageCollection(modis_fires.map(lambda x :ee.Image().constant(x.toDictionary(props).values()).rename(props)))

#print(fire_image.first().getInfo())
# final = composite.addBands(fire_image)
# fire_image.getInfo()

# final.getInfo()
# data = composite.select(bands).sampleRegions(collection=modis_fires,
#                                              properties=['massive'],
#                                              scale=30,
#                                              tileScale=16,
#                                              projection='EPSG:3665',
#                                              region=modis_fires.geometry())
fire_points_train = fire_image.addBands(ee.Image.pixelLonLat()).stratifiedSample(
                                                              numPoints=200,
                                                              classBand='massive',
                                                              scale=30,
                                                              region=modis_fires_train.geometry()).map(
                                                                  lambda f: f.setGeometry(ee.Geometry.Point([f.get('longitude'), f.get('latitude')])))
                                                              
fire_points_test = fire_image.addBands(ee.Image.pixelLonLat()).stratifiedSample(
                                                              numPoints=50,
                                                              classBand='massive',
                                                              scale=30,
                                                              region=modis_fires_test.geometry()).map(
                                                                  lambda f: f.setGeometry(ee.Geometry.Point([f.get('longitude'), f.get('latitude')])))

                                                              



data = composite.sampleRegions(fire_points_train, properties=['massive'], scale=30, geometries=True)
data_test = composite.sampleRegions(fire_points_test, properties=['massive'], scale=30, geometries=True)


def getTempForDate(feature):
  initial_date = feature.get('InitialDate')
  date_filter = ee.Filter.lte('system:time_start', initial_date)
  tempImage = daily_temp.filter(date_filter).limit(1, 'system:time_start', False).first()
  tempImageSample = tempImage.sample(feature.geometry()).first()
  u_wind = ee.Number(tempImageSample.get('u_component_of_wind_10m'))
  v_wind = ee.Number(tempImageSample.get('u_component_of_wind_10m'))
  return feature.set({
      "mean_temp": tempImageSample.get('mean_2m_air_temperature'),
      "wind_speed": (u_wind.pow(2).add(v_wind.pow(2))).sqrt(),
      "date_offset": ee.Number(initial_date).subtract(tempImage.get('system:time_start'))
      })

# def sampleMethod(feature):
#   print(feature.getInfo())
#   return feature

five_days_millis = 432000000
# Get most recent temp data within the past 5 days 
data = data.map(getTempForDate).filter(ee.Filter.lte('date_offset', five_days_millis))
data_test = data_test.map(getTempForDate).filter(ee.Filter.lte('date_offset', five_days_millis))

# massive_fires = modis_fires.filter(ee.Filter.gt('area', massive_fire_threshold)).limit(10).map(lambda f: f.set({'massive': 1})) 



#fire_image.getInfo()
#data.first().getInfo()
#fire_image2.getInfo()
#modis_fires.get('InitialDate').getInfo()

#images = modis_fires.map(
    #lambda f: ee.Image(ee.Number(f.get('InitialDate'))).rename('InitialDate'))

#images.first().getInfo()
#modis_fires.first().getInfo()


# images.getInfo()

In [210]:
print("massive", fire_points_train.filter(ee.Filter.eq('massive', 1)).size().getInfo())
print("small", fire_points_train.filter(ee.Filter.eq('massive', 0)).size().getInfo())

massive 200
small 200


In [211]:
n_train = 0.6
n_val = 0.5
temp = data.filter(ee.Filter.lt('random', 1-n_train))
training = data.filter(ee.Filter.gte('random', 1-n_train))
validation = temp.filter(ee.Filter.lt('random', (1-n_train)*n_val))
testing = temp.filter(ee.Filter.gte('random', (1-n_train)*n_val))

In [218]:
#small_points = composite.sampleRegions(other_fires, geometries=True)
classifier = ee.Classifier.smileRandomForest(100).train(features=data, classProperty='massive', inputProperties=bands)
pred = data_test.classify(classifier)

In [219]:
accuracy = pred.errorMatrix('massive', 'classification')
print(accuracy.getInfo())


[[49, 1], [32, 18]]


In [ ]:
data.first().getInfo()

In [220]:
# pred.errorMatrix('massive', 'classification').accuracy().getInfo()
classifier.explain().getInfo()

{'classes': [0, 1],
 'importance': {'EVC': 13.689297344823537,
  'EVH': 10.990965562998518,
  'EVT': 20.255108520611113,
  'PRS': 24.787762140506192,
  'mean_temp': 70.010267036039,
  'wind_speed': 77.83880510088771},
 'numberOfTrees': 100,
 'outOfBagErrorEstimate': 0.008797653958944282,
 'trees': ['n= 171\nnode), split, n, loss, yval, (yprob)\n* denotes terminal node\n1) root 171 83.041 0 (0.58382 0.41618)\n 2) mean_temp<=293.132 107 32.523 0 (0.80734 0.19266)\n  4) PRS<=1.50000 18 0.0000 1 (0.050000 0.95000) *\n  5) PRS>1.50000 89 3.9101 0 (0.96703 0.032967)\n   10) EVC<=105.500 58 1.9655 0 (0.96667 0.033333)\n    20) mean_temp<=290.358 5 1.6000 0 (0.71429 0.28571)\n     40) EVC<=103.500 4 0.0000 0 (0.83333 0.16667) *\n     41) EVC>103.500 1 0.0000 1 (0.33333 0.66667) *\n    21) mean_temp>290.358 53 0.0000 0 (0.98182 0.018182) *\n   11) EVC>105.500 31 1.9355 0 (0.93939 0.060606)\n    22) PRS<=14.5000 7 1.7143 0 (0.77778 0.22222)\n     44) EVH<=110.500 6 0.0000 0 (0.87500 0.12500) *\n

In [115]:
# Set visualization parameters. I'm pretty sure min and max should track to the min and max values of the set at https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Fire_PRS_v1_2_0#bands
prs_viz_params = {
    'bands': ['PRS']
}

evc_vis_params = {
    'bands': ['EVC'],
    'opacity': 0.5
}
fires_viz_params = {
    'bands': ['massive'],
    'palette': ['#FF0000', '#0000FF'],
     'min': 0,
     'max': 1,
     'opacity': 0.5
}
pred_viz_params = {
    'palette': ['#FF0000', '#0000FF'],
    'min': 0,
    'max': 1,
    'opacity': 0.5
}

# Create a folium map object.
my_map = folium.Map(location=[35.3102, -120.4358], zoom_start=8)

# Add the fire severity & existing vegetation type images to the map object.
my_map.add_ee_layer(landfire_prs, prs_viz_params, 'Percent Replacement-Severity Fire')
my_map.add_ee_layer(landfire_evc, evc_vis_params, 'Existing Vegetation Cover')
my_map.add_ee_layer(fire_image, fires_viz_params, 'Fire Events')
my_map.add_ee_layer(ee.Image().paint(pred, 'classification'), pred_viz_params, 'Predicted Events')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)